# EnergyPlus Building Description Generator

# This notebook reads EnergyPlus IDF model files and extracts detailed building information.
# It can optionally use an LLM to generate human-readable descriptions.

In [ ]:
import os
import re
from pathlib import Path
from collections import defaultdict

def parse_idf_robust(filepath):
    """
    Robust IDF parser that handles all EnergyPlus versions.
    Returns a dictionary of object_type -> list of objects (each object is a list of field values).
    """
    with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
        content = f.read()
    
    # Remove all comments (lines starting with ! or inline comments)
    lines = []
    for line in content.split('\n'):
        # Remove inline comments
        if '!' in line:
            line = line[:line.index('!')]
        lines.append(line)
    content = '\n'.join(lines)
    
    # Split by semicolon to get objects
    raw_objects = content.split(';')
    
    parsed = defaultdict(list)
    for obj in raw_objects:
        obj = obj.strip()
        if not obj:
            continue
        
        # Split by comma and clean up
        fields = [f.strip() for f in obj.split(',')]
        fields = [f for f in fields if f]  # Remove empty fields
        
        if fields:
            obj_type = fields[0]
            obj_values = fields[1:] if len(fields) > 1 else []
            parsed[obj_type].append(obj_values)
    
    return dict(parsed)

# Load .env from parent directory
from dotenv import load_dotenv
load_dotenv(dotenv_path='../.env')

print("IDF Parser loaded successfully!")

IDF Parser loaded successfully!


# List available IDF files
models_dir = Path('models')  # Changed from 'energyplus/models' since we're now in energyplus folder
idf_files = list(models_dir.glob('*.idf'))

print("Available IDF files:")
for i, f in enumerate(idf_files):
    print(f"  {i}: {f.name}")

# Select which file to analyze (change index as needed)
SELECTED_INDEX = 0
idf_path = idf_files[SELECTED_INDEX]
print(f"\nSelected: {idf_path}")

In [45]:
# Load and parse the IDF file
parsed = parse_idf_robust(idf_path)
print(f"Loaded: {idf_path.name}")
print(f"Found {len(parsed)} object types")

# Get version
if 'Version' in parsed and parsed['Version']:
    print(f"IDF Version: {parsed['Version'][0][0]}")

Loaded: ASHRAE901_OfficeMedium_STD2022_Denver.idf
Found 114 object types
IDF Version: 23.2


In [46]:
# Extract Building Information
building_info = {}

# 1. Building object
if 'Building' in parsed and parsed['Building']:
    bldg = parsed['Building'][0]
    building_info['name'] = bldg[0] if bldg else 'Unknown'
    building_info['north_axis'] = bldg[1] if len(bldg) > 1 else 0
    building_info['terrain'] = bldg[2] if len(bldg) > 2 else 'Unknown'
    print(f"Building Name: {building_info['name']}")
    print(f"Terrain: {building_info['terrain']}")

# 2. Site Location
if 'Site:Location' in parsed and parsed['Site:Location']:
    loc = parsed['Site:Location'][0]
    building_info['location'] = {
        'name': loc[0] if loc else 'Unknown',
        'latitude': loc[1] if len(loc) > 1 else 'N/A',
        'longitude': loc[2] if len(loc) > 2 else 'N/A',
        'timezone': loc[3] if len(loc) > 3 else 'N/A',
        'elevation': loc[4] if len(loc) > 4 else 'N/A'
    }
    print(f"\nLocation: {building_info['location']['name']}")
    print(f"  Lat/Long: {building_info['location']['latitude']}, {building_info['location']['longitude']}")
    print(f"  Elevation: {building_info['location']['elevation']} m")

Building Name: OfficeMedium
Terrain: City

Location: Denver-Aurora-Buckley.AFB_CO_USA WMO=724695
  Lat/Long: 39.72, -104.75
  Elevation: 1726.00 m


In [47]:
# 3. Zones - detailed information
zones = parsed.get('Zone', [])
building_info['zones'] = []

print(f"Number of Zones: {len(zones)}\n")
print("Zone Names:")
print("-" * 60)

for zone in zones:
    zone_name = zone[0] if zone else 'Unknown'
    building_info['zones'].append({'name': zone_name})
    print(f"  - {zone_name}")

Number of Zones: 18

Zone Names:
------------------------------------------------------------
  - Core_bottom
  - TopFloor_Plenum
  - MidFloor_Plenum
  - FirstFloor_Plenum
  - Core_mid
  - Core_top
  - Perimeter_top_ZN_3
  - Perimeter_top_ZN_2
  - Perimeter_top_ZN_1
  - Perimeter_top_ZN_4
  - Perimeter_bot_ZN_3
  - Perimeter_bot_ZN_2
  - Perimeter_bot_ZN_1
  - Perimeter_bot_ZN_4
  - Perimeter_mid_ZN_3
  - Perimeter_mid_ZN_2
  - Perimeter_mid_ZN_1
  - Perimeter_mid_ZN_4


In [48]:
# 4. Building Surfaces - Geometry Analysis
surfaces = parsed.get('BuildingSurface:Detailed', [])
fenestrations = parsed.get('FenestrationSurface:Detailed', [])

# Count by surface type (surface type is typically field index 1)
surface_counts = {}
for surf in surfaces:
    if len(surf) > 1:
        stype = surf[1]  # Surface type field
        surface_counts[stype] = surface_counts.get(stype, 0) + 1

building_info['surfaces'] = {
    'total': len(surfaces),
    'by_type': surface_counts,
    'windows': len(fenestrations)
}

print("Building Surfaces:")
print("-" * 40)
print(f"  Total surfaces: {len(surfaces)}")
for stype, count in sorted(surface_counts.items()):
    print(f"    {stype}: {count}")
print(f"  Windows/Doors: {len(fenestrations)}")

Building Surfaces:
----------------------------------------
  Total surfaces: 79
    Ceiling: 15
    Floor: 15
    Roof: 1
    Wall: 48
  Windows/Doors: 24


In [49]:
# 5. HVAC Systems - Detailed Analysis
print("HVAC Systems:")
print("=" * 60)

building_info['hvac'] = {'air_loops': [], 'plant_loops': [], 'components': {}}

# Air Loops
air_loops = parsed.get('AirLoopHVAC', [])
if air_loops:
    print(f"\nAir Loops ({len(air_loops)}):")
    for loop in air_loops:
        name = loop[0] if loop else 'Unknown'
        print(f"  - {name}")
        building_info['hvac']['air_loops'].append(name)

# Plant Loops
plant_loops = parsed.get('PlantLoop', [])
if plant_loops:
    print(f"\nPlant Loops ({len(plant_loops)}):")
    for loop in plant_loops:
        name = loop[0] if loop else 'Unknown'
        print(f"  - {name}")
        building_info['hvac']['plant_loops'].append(name)

# Chillers
chiller_types = ['Chiller:Electric:EIR', 'Chiller:Electric:ReformulatedEIR', 'Chiller:Electric']
for ct in chiller_types:
    chillers = parsed.get(ct, [])
    if chillers:
        print(f"\n{ct} ({len(chillers)}):")
        for ch in chillers:
            print(f"  - {ch[0] if ch else 'Unknown'}")
        building_info['hvac']['components'][ct] = len(chillers)

# Boilers
boilers = parsed.get('Boiler:HotWater', [])
if boilers:
    print(f"\nBoilers ({len(boilers)}):")
    for b in boilers:
        print(f"  - {b[0] if b else 'Unknown'}")
    building_info['hvac']['components']['Boiler:HotWater'] = len(boilers)

# Cooling Coils
coil_types = {
    'Coil:Cooling:DX:SingleSpeed': 'DX Single Speed',
    'Coil:Cooling:DX:TwoSpeed': 'DX Two Speed', 
    'Coil:Cooling:Water': 'Chilled Water',
    'Coil:Cooling:DX:VariableSpeed': 'DX Variable Speed'
}
print(f"\nCooling Coils:")
for ct, label in coil_types.items():
    coils = parsed.get(ct, [])
    if coils:
        print(f"  {label}: {len(coils)}")
        building_info['hvac']['components'][ct] = len(coils)

# Heating Coils
heat_types = {
    'Coil:Heating:Fuel': 'Gas/Fuel',
    'Coil:Heating:Electric': 'Electric',
    'Coil:Heating:Water': 'Hot Water'
}
print(f"\nHeating Coils:")
for ht, label in heat_types.items():
    coils = parsed.get(ht, [])
    if coils:
        print(f"  {label}: {len(coils)}")
        building_info['hvac']['components'][ht] = len(coils)

# Fans
fan_types = {
    'Fan:VariableVolume': 'Variable Volume',
    'Fan:ConstantVolume': 'Constant Volume',
    'Fan:OnOff': 'On/Off',
    'Fan:SystemModel': 'System Model'
}
print(f"\nFans:")
for ft, label in fan_types.items():
    fans = parsed.get(ft, [])
    if fans:
        print(f"  {label}: {len(fans)}")
        building_info['hvac']['components'][ft] = len(fans)

# VAV Terminals
vav_terminals = parsed.get('AirTerminal:SingleDuct:VAV:Reheat', [])
if vav_terminals:
    print(f"\nVAV Terminals with Reheat: {len(vav_terminals)}")
    building_info['hvac']['components']['VAV:Reheat'] = len(vav_terminals)

HVAC Systems:

Air Loops (3):
  - PACU_VAV_bot
  - PACU_VAV_mid
  - PACU_VAV_top

Plant Loops (1):
  - SWHSys1

Cooling Coils:
  DX Two Speed: 3

Heating Coils:
  Gas/Fuel: 3
  Electric: 15

Fans:
  Variable Volume: 3

VAV Terminals with Reheat: 15


In [50]:
# 6. Internal Loads - People, Lights, Equipment
print("Internal Loads:")
print("=" * 60)

# People
people = parsed.get('People', [])
if people:
    print(f"\nPeople Objects ({len(people)}):")
    for p in people[:5]:
        print(f"  - {p[0] if p else 'Unknown'}")
    if len(people) > 5:
        print(f"  ... and {len(people) - 5} more")

# Lights
lights = parsed.get('Lights', [])
if lights:
    print(f"\nLighting Objects ({len(lights)}):")
    for l in lights[:5]:
        print(f"  - {l[0] if l else 'Unknown'}")
    if len(lights) > 5:
        print(f"  ... and {len(lights) - 5} more")

# Electric Equipment
equip = parsed.get('ElectricEquipment', [])
if equip:
    print(f"\nElectric Equipment ({len(equip)}):")
    for e in equip[:5]:
        print(f"  - {e[0] if e else 'Unknown'}")
    if len(equip) > 5:
        print(f"  ... and {len(equip) - 5} more")

building_info['internal_loads'] = {
    'people': len(people),
    'lights': len(lights),
    'equipment': len(equip)
}

Internal Loads:

People Objects (15):
  - Core_bottom
  - Core_mid
  - Core_top
  - Perimeter_top_ZN_3
  - Perimeter_top_ZN_2
  ... and 10 more

Lighting Objects (15):
  - Core_bottom_Lights
  - Core_mid_Lights
  - Core_top_Lights
  - Perimeter_top_ZN_3_Lights
  - Perimeter_top_ZN_2_Lights
  ... and 10 more

Electric Equipment (17):
  - Core_bottom_MiscPlug_Equip
  - Core_mid_MiscPlug_Equip
  - Core_top_MiscPlug_Equip
  - Perimeter_top_ZN_3_MiscPlug_Equip
  - Perimeter_top_ZN_2_MiscPlug_Equip
  ... and 12 more


In [51]:
# 7. Constructions and Materials
print("Constructions & Materials:")
print("=" * 60)

constructions = parsed.get('Construction', [])
materials = parsed.get('Material', [])
materials_nomass = parsed.get('Material:NoMass', [])
windows_simple = parsed.get('WindowMaterial:SimpleGlazingSystem', [])
windows_glazing = parsed.get('WindowMaterial:Glazing', [])

print(f"\nConstructions: {len(constructions)}")
print(f"Materials: {len(materials)} regular, {len(materials_nomass)} no-mass")
print(f"Window Materials: {len(windows_simple)} simple, {len(windows_glazing)} detailed")

# Show some construction names
if constructions:
    print("\nSample Constructions:")
    for c in constructions[:8]:
        print(f"  - {c[0] if c else 'Unknown'}")
    if len(constructions) > 8:
        print(f"  ... and {len(constructions) - 8} more")

building_info['envelope'] = {
    'constructions': len(constructions),
    'materials': len(materials) + len(materials_nomass),
    'window_materials': len(windows_simple) + len(windows_glazing)
}

Constructions & Materials:

Constructions: 38
Materials: 18 regular, 19 no-mass
Window Materials: 4 simple, 0 detailed

Sample Constructions:
  - Swinging Door_con
  - InteriorFurnishings
  - Air_Wall
  - DropCeiling
  - OpaqueDoor
  - AtticRoofDeck
  - int_wall
  - ext_slab_8in_with_carpet
  ... and 30 more


In [ ]:
# 7b. ENVELOPE THERMAL PROPERTIES - U-Values and R-Values
print("Envelope Thermal Properties:")
print("=" * 70)

building_info['thermal_properties'] = {'materials': {}, 'windows': [], 'constructions': {}}

# Parse Material properties (conductivity, thickness -> calculate R-value)
# Material fields: Name, Roughness, Thickness(m), Conductivity(W/m-K), Density, Specific Heat
materials = parsed.get('Material', [])
print("\nMaterial Properties:")
print("-" * 70)
print(f"{'Material Name':<40} {'Thickness(m)':<12} {'Conductivity':<12} {'R-Value':<10}")
print("-" * 70)

for mat in materials[:15]:  # Show first 15
    if len(mat) >= 4:
        name = mat[0]
        try:
            thickness = float(mat[2])  # meters
            conductivity = float(mat[3])  # W/m-K
            r_value = thickness / conductivity if conductivity > 0 else 0
            building_info['thermal_properties']['materials'][name] = {
                'thickness_m': thickness,
                'conductivity': conductivity,
                'r_value_m2K_W': round(r_value, 4)
            }
            print(f"{name:<40} {thickness:<12.4f} {conductivity:<12.4f} {r_value:<10.4f}")
        except (ValueError, IndexError):
            pass

if len(materials) > 15:
    print(f"... and {len(materials) - 15} more materials")

# Parse Material:NoMass (direct R-value)
# Fields: Name, Roughness, Thermal Resistance (R-value in m2-K/W)
materials_nomass = parsed.get('Material:NoMass', [])
if materials_nomass:
    print("\n\nNo-Mass Materials (Insulation Layers):")
    print("-" * 70)
    print(f"{'Material Name':<50} {'R-Value (m²·K/W)':<15}")
    print("-" * 70)
    
    for mat in materials_nomass[:10]:
        if len(mat) >= 3:
            name = mat[0]
            try:
                r_value = float(mat[2])
                building_info['thermal_properties']['materials'][name] = {
                    'r_value_m2K_W': round(r_value, 4),
                    'type': 'no-mass'
                }
                print(f"{name:<50} {r_value:<15.4f}")
            except (ValueError, IndexError):
                pass
    
    if len(materials_nomass) > 10:
        print(f"... and {len(materials_nomass) - 10} more")

# Parse WindowMaterial:SimpleGlazingSystem
# Fields: Name, U-Factor (W/m2-K), SHGC
windows_simple = parsed.get('WindowMaterial:SimpleGlazingSystem', [])
if windows_simple:
    print("\n\nWindow Properties (Simple Glazing):")
    print("-" * 70)
    print(f"{'Window Name':<40} {'U-Factor (W/m²·K)':<18} {'SHGC':<10}")
    print("-" * 70)
    
    for win in windows_simple:
        if len(win) >= 3:
            name = win[0]
            try:
                u_factor = float(win[1])
                shgc = float(win[2])
                building_info['thermal_properties']['windows'].append({
                    'name': name,
                    'u_factor': u_factor,
                    'shgc': shgc
                })
                print(f"{name:<40} {u_factor:<18.3f} {shgc:<10.3f}")
            except (ValueError, IndexError):
                pass

In [ ]:
# 7c. HVAC SYSTEM EFFICIENCIES
print("HVAC System Efficiencies:")
print("=" * 70)

building_info['efficiencies'] = {'cooling': [], 'heating': [], 'fans': [], 'boilers': [], 'chillers': []}

# DX Cooling Coils - COP/EER
# Coil:Cooling:DX:TwoSpeed fields include: Name, ..., Rated High Speed COP, Rated Low Speed COP
dx_coils_two = parsed.get('Coil:Cooling:DX:TwoSpeed', [])
if dx_coils_two:
    print("\nDX Cooling Coils (Two-Speed):")
    print("-" * 70)
    print(f"{'Coil Name':<45} {'High COP':<12} {'Low COP':<12}")
    print("-" * 70)
    for coil in dx_coils_two:
        if len(coil) >= 10:
            name = coil[0]
            try:
                # Field positions vary by version, search for COP values
                high_cop = coil[6] if len(coil) > 6 else 'N/A'
                low_cop = coil[14] if len(coil) > 14 else 'N/A'
                building_info['efficiencies']['cooling'].append({
                    'name': name,
                    'type': 'DX Two-Speed',
                    'high_cop': high_cop,
                    'low_cop': low_cop
                })
                print(f"{name:<45} {str(high_cop):<12} {str(low_cop):<12}")
            except:
                print(f"{name:<45} {'N/A':<12} {'N/A':<12}")

# DX Single Speed
dx_coils_single = parsed.get('Coil:Cooling:DX:SingleSpeed', [])
if dx_coils_single:
    print("\nDX Cooling Coils (Single-Speed):")
    print("-" * 70)
    for coil in dx_coils_single:
        if coil:
            name = coil[0]
            cop = coil[5] if len(coil) > 5 else 'N/A'
            building_info['efficiencies']['cooling'].append({
                'name': name,
                'type': 'DX Single-Speed',
                'cop': cop
            })
            print(f"  {name}: COP = {cop}")

# Gas/Fuel Heating Coils - Efficiency
# Coil:Heating:Fuel fields: Name, Schedule, Fuel Type, Burner Efficiency, Capacity
heating_fuel = parsed.get('Coil:Heating:Fuel', [])
if heating_fuel:
    print("\nGas/Fuel Heating Coils:")
    print("-" * 70)
    print(f"{'Coil Name':<45} {'Fuel Type':<15} {'Efficiency':<12}")
    print("-" * 70)
    for coil in heating_fuel:
        if len(coil) >= 4:
            name = coil[0]
            fuel_type = coil[2] if len(coil) > 2 else 'N/A'
            efficiency = coil[3] if len(coil) > 3 else 'N/A'
            building_info['efficiencies']['heating'].append({
                'name': name,
                'fuel_type': fuel_type,
                'efficiency': efficiency
            })
            print(f"{name:<45} {fuel_type:<15} {str(efficiency):<12}")

# Boilers
boilers = parsed.get('Boiler:HotWater', [])
if boilers:
    print("\nBoilers:")
    print("-" * 70)
    print(f"{'Boiler Name':<40} {'Fuel':<12} {'Efficiency':<12} {'Capacity':<15}")
    print("-" * 70)
    for boiler in boilers:
        if len(boiler) >= 4:
            name = boiler[0]
            fuel = boiler[1] if len(boiler) > 1 else 'N/A'
            efficiency = boiler[2] if len(boiler) > 2 else 'N/A'
            capacity = boiler[3] if len(boiler) > 3 else 'autosize'
            building_info['efficiencies']['boilers'].append({
                'name': name,
                'fuel': fuel,
                'efficiency': efficiency,
                'capacity': capacity
            })
            print(f"{name:<40} {fuel:<12} {str(efficiency):<12} {str(capacity):<15}")

# Chillers
for chiller_type in ['Chiller:Electric:EIR', 'Chiller:Electric:ReformulatedEIR']:
    chillers = parsed.get(chiller_type, [])
    if chillers:
        print(f"\n{chiller_type}:")
        print("-" * 70)
        for ch in chillers:
            if ch:
                name = ch[0]
                cop = ch[2] if len(ch) > 2 else 'N/A'
                building_info['efficiencies']['chillers'].append({
                    'name': name,
                    'type': chiller_type,
                    'cop': cop
                })
                print(f"  {name}: Reference COP = {cop}")

# Fan Efficiencies
# Fan:VariableVolume fields: Name, Schedule, Fan Total Efficiency, Pressure Rise, ...
fans_vav = parsed.get('Fan:VariableVolume', [])
if fans_vav:
    print("\nVariable Volume Fans:")
    print("-" * 70)
    print(f"{'Fan Name':<40} {'Efficiency':<12} {'Pressure Rise (Pa)':<18}")
    print("-" * 70)
    for fan in fans_vav:
        if len(fan) >= 4:
            name = fan[0]
            efficiency = fan[2] if len(fan) > 2 else 'N/A'
            pressure = fan[3] if len(fan) > 3 else 'N/A'
            building_info['efficiencies']['fans'].append({
                'name': name,
                'type': 'VAV',
                'efficiency': efficiency,
                'pressure_rise': pressure
            })
            print(f"{name:<40} {str(efficiency):<12} {str(pressure):<18}")

In [ ]:
# 7d. INFILTRATION RATES
print("Infiltration & Ventilation:")
print("=" * 70)

building_info['infiltration'] = []
building_info['ventilation'] = []

# ZoneInfiltration:DesignFlowRate
# Fields: Name, Zone, Schedule, Design Flow Rate Calculation Method, Design Flow Rate, Flow per Zone Area, ...
infiltration = parsed.get('ZoneInfiltration:DesignFlowRate', [])
if infiltration:
    print("\nZone Infiltration (Design Flow Rate):")
    print("-" * 70)
    print(f"{'Name':<35} {'Zone':<25} {'Method':<20} {'Value':<15}")
    print("-" * 70)
    
    for inf in infiltration:
        if len(inf) >= 4:
            name = inf[0]
            zone = inf[1] if len(inf) > 1 else 'N/A'
            method = inf[3] if len(inf) > 3 else 'N/A'
            
            # Get the appropriate value based on method
            if 'Flow/Zone' in method or 'Flow/Area' in method:
                value = inf[5] if len(inf) > 5 else 'N/A'
                unit = 'm³/s/m²'
            elif 'Flow/ExteriorArea' in method:
                value = inf[6] if len(inf) > 6 else 'N/A'
                unit = 'm³/s/m²'
            elif 'AirChanges' in method:
                value = inf[7] if len(inf) > 7 else 'N/A'
                unit = 'ACH'
            else:
                value = inf[4] if len(inf) > 4 else 'N/A'
                unit = 'm³/s'
            
            building_info['infiltration'].append({
                'name': name,
                'zone': zone,
                'method': method,
                'value': value,
                'unit': unit
            })
            print(f"{name:<35} {zone:<25} {method:<20} {str(value):<15}")

# ZoneInfiltration:EffectiveLeakageArea (if present)
infiltration_ela = parsed.get('ZoneInfiltration:EffectiveLeakageArea', [])
if infiltration_ela:
    print("\nZone Infiltration (Effective Leakage Area):")
    print("-" * 70)
    for inf in infiltration_ela:
        if inf:
            name = inf[0]
            zone = inf[1] if len(inf) > 1 else 'N/A'
            ela = inf[3] if len(inf) > 3 else 'N/A'
            print(f"  {name}: Zone={zone}, ELA={ela} cm²")

# DesignSpecification:OutdoorAir (ventilation requirements)
outdoor_air = parsed.get('DesignSpecification:OutdoorAir', [])
if outdoor_air:
    print("\nOutdoor Air Specifications:")
    print("-" * 70)
    print(f"{'Name':<40} {'Method':<25} {'Flow/Person':<15} {'Flow/Area':<15}")
    print("-" * 70)
    
    for oa in outdoor_air[:10]:
        if len(oa) >= 4:
            name = oa[0]
            method = oa[1] if len(oa) > 1 else 'N/A'
            flow_person = oa[2] if len(oa) > 2 else 'N/A'
            flow_area = oa[3] if len(oa) > 3 else 'N/A'
            building_info['ventilation'].append({
                'name': name,
                'method': method,
                'flow_per_person': flow_person,
                'flow_per_area': flow_area
            })
            print(f"{name:<40} {method:<25} {str(flow_person):<15} {str(flow_area):<15}")
    
    if len(outdoor_air) > 10:
        print(f"... and {len(outdoor_air) - 10} more")

# ZoneVentilation:DesignFlowRate (if present)
zone_vent = parsed.get('ZoneVentilation:DesignFlowRate', [])
if zone_vent:
    print("\nZone Ventilation (Design Flow Rate):")
    print("-" * 70)
    for vent in zone_vent[:5]:
        if vent:
            name = vent[0]
            zone = vent[1] if len(vent) > 1 else 'N/A'
            print(f"  {name}: Zone={zone}")
    if len(zone_vent) > 5:
        print(f"  ... and {len(zone_vent) - 5} more")

In [52]:
# 8. Schedules Summary
print("Schedules:")
print("=" * 60)

schedule_compact = parsed.get('Schedule:Compact', [])
schedule_constant = parsed.get('Schedule:Constant', [])
schedule_file = parsed.get('Schedule:File', [])
schedule_year = parsed.get('Schedule:Year', [])

print(f"\nSchedule:Compact: {len(schedule_compact)}")
print(f"Schedule:Constant: {len(schedule_constant)}")
print(f"Schedule:Year: {len(schedule_year)}")
print(f"Schedule:File: {len(schedule_file)}")

# Show some schedule names
all_schedules = schedule_compact + schedule_constant
if all_schedules:
    print("\nSample Schedules:")
    for s in all_schedules[:10]:
        print(f"  - {s[0] if s else 'Unknown'}")
    if len(all_schedules) > 10:
        print(f"  ... and {len(all_schedules) - 10} more")

building_info['schedules'] = {
    'compact': len(schedule_compact),
    'constant': len(schedule_constant),
    'year': len(schedule_year),
    'file': len(schedule_file),
    'total': len(schedule_compact) + len(schedule_constant) + len(schedule_year) + len(schedule_file)
}

Schedules:

Schedule:Compact: 123
Schedule:Constant: 3
Schedule:Year: 0
Schedule:File: 0

Sample Schedules:
  - PV_SCH
  - Exterior_lighting_schedule_a
  - Exterior_lighting_schedule_b
  - Exterior_lighting_schedule_a_IECC2015
  - Exterior_lighting_schedule_b_2016
  - Exterior_lighting_schedule_c_2016
  - Hours_of_operation
  - ALWAYS_ON
  - Exterior_Lgt_ALWAYS_ON
  - Exterior_Lgt_189_1
  ... and 116 more


In [53]:
# 9. Simulation Period
print("Simulation Settings:")
print("=" * 60)

run_periods = parsed.get('RunPeriod', [])
if run_periods:
    rp = run_periods[0]
    print(f"\nRun Period: {rp[0] if rp else 'Unknown'}")
    if len(rp) >= 5:
        print(f"  Start: {rp[1]}/{rp[2]}")
        print(f"  End: {rp[3]}/{rp[4]}")
        building_info['run_period'] = {
            'start': f"{rp[1]}/{rp[2]}",
            'end': f"{rp[3]}/{rp[4]}"
        }

# Timestep
timesteps = parsed.get('Timestep', [])
if timesteps and timesteps[0]:
    ts = timesteps[0][0]
    print(f"\nTimesteps per Hour: {ts}")
    building_info['timestep'] = ts

Simulation Settings:

Run Period: RUNPERIOD 1
  Start: 1/1
  End: 12/31

Timesteps per Hour: 6


In [54]:
# 10. Complete Building Summary
print("=" * 70)
print("BUILDING SUMMARY")
print("=" * 70)

print(f"""
Building: {building_info.get('name', 'Unknown')}
Location: {building_info.get('location', {}).get('name', 'Not specified')}

GEOMETRY:
  Zones: {len(building_info.get('zones', []))}
  Surfaces: {building_info.get('surfaces', {}).get('total', 0)}
  Windows/Doors: {building_info.get('surfaces', {}).get('windows', 0)}

HVAC:
  Air Loops: {len(building_info.get('hvac', {}).get('air_loops', []))}
  Plant Loops: {len(building_info.get('hvac', {}).get('plant_loops', []))}
  Components: {building_info.get('hvac', {}).get('components', {})}

INTERNAL LOADS:
  People objects: {building_info.get('internal_loads', {}).get('people', 0)}
  Lighting objects: {building_info.get('internal_loads', {}).get('lights', 0)}
  Equipment objects: {building_info.get('internal_loads', {}).get('equipment', 0)}

ENVELOPE:
  Constructions: {building_info.get('envelope', {}).get('constructions', 0)}
  Materials: {building_info.get('envelope', {}).get('materials', 0)}

SCHEDULES:
  Total: {building_info.get('schedules', {}).get('total', 0)}
""")

print("\n[Building info stored in 'building_info' dictionary for LLM prompt generation]")

BUILDING SUMMARY

Building: OfficeMedium
Location: Denver-Aurora-Buckley.AFB_CO_USA WMO=724695

GEOMETRY:
  Zones: 18
  Surfaces: 79
  Windows/Doors: 24

HVAC:
  Air Loops: 3
  Plant Loops: 1
  Components: {'Coil:Cooling:DX:TwoSpeed': 3, 'Coil:Heating:Fuel': 3, 'Coil:Heating:Electric': 15, 'Fan:VariableVolume': 3, 'VAV:Reheat': 15}

INTERNAL LOADS:
  People objects: 15
  Lighting objects: 15
  Equipment objects: 17

ENVELOPE:
  Constructions: 38
  Materials: 37

SCHEDULES:
  Total: 126


[Building info stored in 'building_info' dictionary for LLM prompt generation]


## Save Building Info to Text File

Run this cell to save the extracted building information to a formatted text file before sending to the LLM.

In [ ]:
# Save extracted building info to a formatted text file
from datetime import datetime

def save_building_info_to_file(building_info, idf_path, output_dir='../outputs'):  # Changed to parent outputs folder
    """Save building information to a formatted text file."""
    
    # Create output directory if it doesn't exist
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Generate filename based on IDF name
    idf_name = Path(idf_path).stem
    filename = output_path / f"{idf_name}_building_info.txt"
    
    # Format zone names
    zone_names = [z['name'] for z in building_info.get('zones', [])]
    zones_str = "\n    ".join(zone_names) if zone_names else "None"
    
    # Format HVAC components
    hvac_components = building_info.get('hvac', {}).get('components', {})
    hvac_str = "\n    ".join([f"{k}: {v}" for k, v in hvac_components.items()]) if hvac_components else "None"
    
    # Format surface types
    surface_types = building_info.get('surfaces', {}).get('by_type', {})
    surfaces_str = "\n    ".join([f"{k}: {v}" for k, v in surface_types.items()]) if surface_types else "None"
    
    # Format thermal properties - materials
    materials_str = ""
    for name, props in list(building_info.get('thermal_properties', {}).get('materials', {}).items())[:20]:
        if 'conductivity' in props:
            materials_str += f"    {name}: k={props['conductivity']} W/m·K, R={props['r_value_m2K_W']} m²·K/W\n"
        else:
            materials_str += f"    {name}: R={props['r_value_m2K_W']} m²·K/W (no-mass)\n"
    if not materials_str:
        materials_str = "    None extracted"
    
    # Format window properties
    windows_str = ""
    for win in building_info.get('thermal_properties', {}).get('windows', []):
        windows_str += f"    {win['name']}: U={win['u_factor']} W/m²·K, SHGC={win['shgc']}\n"
    if not windows_str:
        windows_str = "    None extracted"
    
    # Format efficiencies - cooling
    cooling_eff_str = ""
    for eff in building_info.get('efficiencies', {}).get('cooling', []):
        if 'high_cop' in eff:
            cooling_eff_str += f"    {eff['name']}: High COP={eff['high_cop']}, Low COP={eff['low_cop']}\n"
        else:
            cooling_eff_str += f"    {eff['name']}: COP={eff.get('cop', 'N/A')}\n"
    if not cooling_eff_str:
        cooling_eff_str = "    None extracted"
    
    # Format efficiencies - heating
    heating_eff_str = ""
    for eff in building_info.get('efficiencies', {}).get('heating', []):
        heating_eff_str += f"    {eff['name']}: Fuel={eff.get('fuel_type', 'N/A')}, Efficiency={eff.get('efficiency', 'N/A')}\n"
    if not heating_eff_str:
        heating_eff_str = "    None extracted"
    
    # Format efficiencies - fans
    fan_eff_str = ""
    for eff in building_info.get('efficiencies', {}).get('fans', []):
        fan_eff_str += f"    {eff['name']}: Efficiency={eff.get('efficiency', 'N/A')}, Pressure Rise={eff.get('pressure_rise', 'N/A')} Pa\n"
    if not fan_eff_str:
        fan_eff_str = "    None extracted"
    
    # Format infiltration
    infiltration_str = ""
    for inf in building_info.get('infiltration', []):
        infiltration_str += f"    {inf['zone']}: {inf['value']} {inf.get('unit', '')} ({inf['method']})\n"
    if not infiltration_str:
        infiltration_str = "    None extracted"
    
    # Format ventilation
    ventilation_str = ""
    for vent in building_info.get('ventilation', [])[:10]:
        ventilation_str += f"    {vent['name']}: {vent['flow_per_person']} m³/s/person, {vent['flow_per_area']} m³/s/m²\n"
    if not ventilation_str:
        ventilation_str = "    None extracted"
    
    content = f"""================================================================================
ENERGYPLUS BUILDING MODEL INFORMATION
================================================================================
Generated: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
Source File: {idf_path}

================================================================================
1. BUILDING IDENTIFICATION
================================================================================
Building Name: {building_info.get('name', 'Unknown')}
Terrain: {building_info.get('terrain', 'Unknown')}
North Axis: {building_info.get('north_axis', 0)}°

================================================================================
2. LOCATION
================================================================================
Name: {building_info.get('location', {}).get('name', 'Not specified')}
Latitude: {building_info.get('location', {}).get('latitude', 'N/A')}
Longitude: {building_info.get('location', {}).get('longitude', 'N/A')}
Time Zone: {building_info.get('location', {}).get('timezone', 'N/A')}
Elevation: {building_info.get('location', {}).get('elevation', 'N/A')} m

================================================================================
3. THERMAL ZONES ({len(zone_names)})
================================================================================
    {zones_str}

================================================================================
4. BUILDING GEOMETRY
================================================================================
Total Surfaces: {building_info.get('surfaces', {}).get('total', 0)}
Windows/Doors: {building_info.get('surfaces', {}).get('windows', 0)}

Surface Types:
    {surfaces_str}

================================================================================
5. ENVELOPE THERMAL PROPERTIES
================================================================================
Material Thermal Properties:
{materials_str}
Window Properties (U-Factor & SHGC):
{windows_str}

================================================================================
6. HVAC SYSTEMS
================================================================================
Air Loops ({len(building_info.get('hvac', {}).get('air_loops', []))}):
    {chr(10).join(['    ' + x for x in building_info.get('hvac', {}).get('air_loops', [])]) or '    None'}

Plant Loops ({len(building_info.get('hvac', {}).get('plant_loops', []))}):
    {chr(10).join(['    ' + x for x in building_info.get('hvac', {}).get('plant_loops', [])]) or '    None'}

HVAC Components:
    {hvac_str}

================================================================================
7. HVAC SYSTEM EFFICIENCIES
================================================================================
Cooling Equipment:
{cooling_eff_str}
Heating Equipment:
{heating_eff_str}
Fan Efficiencies:
{fan_eff_str}

================================================================================
8. INFILTRATION & VENTILATION
================================================================================
Zone Infiltration Rates:
{infiltration_str}
Outdoor Air Specifications:
{ventilation_str}

================================================================================
9. INTERNAL LOADS
================================================================================
People Objects: {building_info.get('internal_loads', {}).get('people', 0)}
Lighting Objects: {building_info.get('internal_loads', {}).get('lights', 0)}
Electric Equipment Objects: {building_info.get('internal_loads', {}).get('equipment', 0)}

================================================================================
10. BUILDING ENVELOPE SUMMARY
================================================================================
Constructions: {building_info.get('envelope', {}).get('constructions', 0)}
Materials: {building_info.get('envelope', {}).get('materials', 0)}
Window Materials: {building_info.get('envelope', {}).get('window_materials', 0)}

================================================================================
11. SCHEDULES
================================================================================
Schedule:Compact: {building_info.get('schedules', {}).get('compact', 0)}
Schedule:Constant: {building_info.get('schedules', {}).get('constant', 0)}
Schedule:Year: {building_info.get('schedules', {}).get('year', 0)}
Schedule:File: {building_info.get('schedules', {}).get('file', 0)}
Total Schedules: {building_info.get('schedules', {}).get('total', 0)}

================================================================================
12. SIMULATION SETTINGS
================================================================================
Run Period: {building_info.get('run_period', {}).get('start', 'N/A')} to {building_info.get('run_period', {}).get('end', 'N/A')}
Timesteps per Hour: {building_info.get('timestep', 'N/A')}

================================================================================
END OF BUILDING INFORMATION
================================================================================
"""
    
    # Write to file
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(content)
    
    print(f"Building information saved to: {filename}")
    return filename, content

# Save the building info
output_file, file_content = save_building_info_to_file(building_info, idf_path)
print("\n" + "=" * 70)
print("FILE PREVIEW:")
print("=" * 70)
print(file_content[:3000] + "..." if len(file_content) > 3000 else file_content)

Building information saved to: outputs\ASHRAE901_OfficeMedium_STD2022_Denver_building_info.txt

FILE PREVIEW:
ENERGYPLUS BUILDING MODEL INFORMATION
Generated: 2026-02-01 20:10:55
Source File: energyplus\models\ASHRAE901_OfficeMedium_STD2022_Denver.idf

1. BUILDING IDENTIFICATION
Building Name: OfficeMedium
Terrain: City
North Axis: 0°

2. LOCATION
Name: Denver-Aurora-Buckley.AFB_CO_USA WMO=724695
Latitude: 39.72
Longitude: -104.75
Time Zone: -7.00
Elevation: 1726.00 m

3. THERMAL ZONES (18)
    Core_bottom
    TopFloor_Plenum
    MidFloor_Plenum
    FirstFloor_Plenum
    Core_mid
    Core_top
    Perimeter_top_ZN_3
    Perimeter_top_ZN_2
    Perimeter_top_ZN_1
    Perimeter_top_ZN_4
    Perimeter_bot_ZN_3
    Perimeter_bot_ZN_2
    Perimeter_bot_ZN_1
    Perimeter_bot_ZN_4
    Perimeter_mid_ZN_3
    Perimeter_mid_ZN_2
    Perimeter_mid_ZN_1
    Perimeter_mid_ZN_4

4. BUILDING GEOMETRY
Total Surfaces: 79
Windows/Doors: 24

Surface Types:
    Roof: 1
    Ceiling: 15
    Floor: 15
    Wal

## Generate LLM Description (Optional)

Run the cell below to send the extracted building information to OpenAI for a natural language description.

In [ ]:
from dotenv import load_dotenv
from openai import OpenAI

# Load API key from .env file in parent directory
load_dotenv(dotenv_path='../.env')

def get_building_description_from_file(file_content, model="gpt-4o-mini"):
    """Call OpenAI API to generate building description from the saved text file content."""
    client = OpenAI()
    
    prompt = f"""You are an expert in building energy modeling and EnergyPlus simulations.

Based on the following EnergyPlus model information, provide a comprehensive but concise description of this building.

{file_content}

Please describe:
1. Building type and likely use case
2. Size and layout based on zones
3. HVAC system type and configuration
4. Notable features
5. Model complexity assessment
"""
    
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are an expert building energy analyst who provides clear, technical descriptions of buildings based on EnergyPlus model data."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=1500
    )
    return response.choices[0].message.content

# Generate and display description using the saved file content
print("Generating LLM description from saved building info...\n")
description = get_building_description_from_file(file_content)
print("=" * 70)
print("AI-GENERATED BUILDING DESCRIPTION")
print("=" * 70)
print(description)

# Optionally save the description too
desc_file = output_file.parent / f"{Path(idf_path).stem}_llm_description.txt"
with open(desc_file, 'w', encoding='utf-8') as f:
    f.write(f"AI-Generated Building Description\n")
    f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Source: {idf_path}\n")
    f.write("=" * 70 + "\n\n")
    f.write(description)
print(f"\nDescription saved to: {desc_file}")